In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

In [2]:
import logging

# Set the logging level to ERROR
logging.getLogger("py4j").setLevel(logging.ERROR)

In [3]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Preprocess") \
    .config("spark.executor.memory", "18g") \
    .config("spark.driver.memory", "150g") \
    .config("spark.pyspark.python", "/usr/local/bin/python3.9") \
    .config("spark.pyspark.driver.python", "/usr/local/bin/python3.9") \
    .getOrCreate()

# Read CSV into DataFrame
# prices = spark.read.csv("itineraries.csv", header=True, inferSchema=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/09 16:13:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/09 16:13:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/09 16:13:58 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
flight_prices = spark.read.csv("itineraries.csv", header=True, inferSchema=True)

24/05/09 16:14:16 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [42]:
flight_prices.columns

['legId',
 'searchDate',
 'flightDate',
 'startingAirport',
 'destinationAirport',
 'fareBasisCode',
 'travelDuration',
 'elapsedDays',
 'isBasicEconomy',
 'isRefundable',
 'isNonStop',
 'baseFare',
 'totalFare',
 'seatsRemaining',
 'totalTravelDistance',
 'segmentsDepartureTimeEpochSeconds',
 'segmentsDepartureTimeRaw',
 'segmentsArrivalTimeEpochSeconds',
 'segmentsArrivalTimeRaw',
 'segmentsArrivalAirportCode',
 'segmentsDepartureAirportCode',
 'segmentsAirlineName',
 'segmentsAirlineCode',
 'segmentsEquipmentDescription',
 'segmentsDurationInSeconds',
 'segmentsDistance',
 'segmentsCabinCode']

In [6]:
flight_prices.select("flightDate").count()

82138753

In [43]:
from pyspark.sql.functions import split, explode

# Split the flight_name column by '||'
flight_prices_unique = flight_prices.withColumn("segmentsAirlineName_array", split(flight_prices["segmentsAirlineName"], "\|\|"))

# Explode the array into multiple rows
flight_prices_unique = flight_prices_unique.select(explode(flight_prices_unique["segmentsAirlineName_array"]).alias("flight_name"))

# Get unique flight names
unique_flights = flight_prices_unique.distinct()

# Show the result
unique_flights.show()

+--------------------+
|         flight_name|
+--------------------+
|               Delta|
|        Boutique Air|
|        Key Lime Air|
|Sun Country Airlines|
|    Contour Airlines|
|              United|
|            Cape Air|
|   Frontier Airlines|
|   American Airlines|
|     JetBlue Airways|
|     Alaska Airlines|
|     Spirit Airlines|
|Southern Airways ...|
|   Hawaiian Airlines|
|      Silver Airways|
+--------------------+



In [44]:
from pyspark.sql.functions import col
filtered_df = flight_prices.filter(col("segmentsArrivalAirportCode").like("%||%"))
filtered_df.limit(10).toPandas().head(10)

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,9335fae376c38bb61263281779f469ec,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT4H12M,0,False,False,...,1650203400||1650213120,2022-04-17T09:50:00.000-04:00||2022-04-17T12:3...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A320||Airbus A320,5400||7500,228||728,coach||coach
1,3904bf87f2d1daf334f1ae7e3b876028,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT5H18M,0,False,False,...,1650203400||1650217080,2022-04-17T09:50:00.000-04:00||2022-04-17T13:3...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A320||Boeing 737-800,5400||8280,228||728,coach||coach
2,d93988734c44a3c075d9efe373352507,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT5H32M,0,False,False,...,1650198060||1650213120,2022-04-17T08:21:00.000-04:00||2022-04-17T12:3...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A319||Airbus A320,4860||7500,228||728,coach||coach
3,562e7d5dd6ecbf1509c0c19711dbdca9,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT6H38M,0,False,False,...,1650198060||1650217080,2022-04-17T08:21:00.000-04:00||2022-04-17T13:3...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A319||Boeing 737-800,4860||8280,228||728,coach||coach
4,c38a6e4b807d15541e5866676febcbec,2022-04-16,2022-04-17,ATL,BOS,VAA0AKEN,PT4H46M,0,False,False,...,1650194280||1650204960,2022-04-17T07:18:00.000-04:00||2022-04-17T10:1...,IAD||BOS,ATL||IAD,United||United,UA||UA,Airbus A319||Boeing 737-800,6480||5940,541||406,coach||coach
5,f66d72ba3a526576608b8402b5720726,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT5H45M,0,False,False,...,1650228840||1650241440,2022-04-17T15:54:00.000-05:00||2022-04-17T20:2...,ORD||BOS,ATL||ORD,American Airlines||American Airlines,AA||AA,Embraer 175||Boeing 737-800,8100||8340,600||862,coach||coach
6,e7c4054e85cca9bc7134b3cc93deaf7c,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT5H59M,0,False,False,...,1650240660||1650254340,2022-04-17T19:11:00.000-05:00||2022-04-17T23:5...,ORD||BOS,ATL||ORD,American Airlines||American Airlines,AA||AA,Embraer 175||Boeing 737-800,7860||8640,600||862,coach||coach
7,5fa8c0f8b25eb24bf2ddc39a498bcc37,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT7H18M,0,False,False,...,1650228840||1650247020,2022-04-17T15:54:00.000-05:00||2022-04-17T21:5...,ORD||BOS,ATL||ORD,American Airlines||American Airlines,AA||AA,Embraer 175||Airbus A321,8100||7920,600||862,coach||coach
8,948d26b3e5658762cd62f842c87ae46e,2022-04-16,2022-04-17,ATL,BOS,VAA0AKEN,PT8H10M,0,False,False,...,1650198000||1650219600,2022-04-17T07:20:00.000-05:00||2022-04-17T14:2...,ORD||BOS,ATL||ORD,United||United,UA||UA,Embraer 175 (Enhanced Winglets)||Boeing 737 MAX 9,7800||8640,600||862,coach||coach
9,cb345693479e84838dfaac346b8e8d6a,2022-04-16,2022-04-17,ATL,BOS,VNR,PT5H39M,0,False,False,...,1650194400||1650207900,2022-04-17T07:20:00.000-04:00||2022-04-17T11:0...,FLL||BOS,ATL||FLL,Spirit Airlines||Spirit Airlines,NK||NK,AIRBUS INDUSTRIE A321 SHARKLETS||,6840||11100,None||None,coach||coach


In [45]:
columns = [
    "searchDate",
    "flightDate",
    "startingAirport",
    "destinationAirport",
    "travelDuration",
    "isRefundable",
    "isNonStop",
    "totalTravelDistance",
    "segmentsDepartureTimeRaw",
    "segmentsArrivalTimeRaw",
    "segmentsAirlineName",
    "segmentsArrivalAirportCode",
    "segmentsDistance",
    "totalFare"
]


flight_prices_processed = flight_prices[columns]

In [46]:
flight_prices_processed.limit(10).toPandas().head(10)

,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isRefundable,isNonStop,totalTravelDistance,segmentsDepartureTimeRaw,segmentsArrivalTimeRaw,segmentsAirlineName,segmentsArrivalAirportCode,segmentsDistance,totalFare
0,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,True,947,2022-04-17T12:57:00.000-04:00,2022-04-17T15:26:00.000-04:00,Delta,BOS,947,248.6
1,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,True,947,2022-04-17T06:30:00.000-04:00,2022-04-17T09:00:00.000-04:00,Delta,BOS,947,248.6
2,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,True,947,2022-04-17T11:35:00.000-04:00,2022-04-17T14:05:00.000-04:00,Delta,BOS,947,248.6
3,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,True,947,2022-04-17T13:59:00.000-04:00,2022-04-17T16:31:00.000-04:00,Delta,BOS,947,248.6
4,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,True,947,2022-04-17T09:59:00.000-04:00,2022-04-17T12:33:00.000-04:00,Delta,BOS,947,248.6
5,2022-04-16,2022-04-17,ATL,BOS,PT2H38M,False,True,947,2022-04-17T10:45:00.000-04:00,2022-04-17T13:23:00.000-04:00,JetBlue Airways,BOS,947,248.6
6,2022-04-16,2022-04-17,ATL,BOS,PT4H12M,False,False,956,2022-04-17T08:20:00.000-04:00||2022-04-17T10:2...,2022-04-17T09:50:00.000-04:00||2022-04-17T12:3...,American Airlines||American Airlines,CLT||BOS,228||728,251.1
7,2022-04-16,2022-04-17,ATL,BOS,PT5H18M,False,False,956,2022-04-17T08:20:00.000-04:00||2022-04-17T11:2...,2022-04-17T09:50:00.000-04:00||2022-04-17T13:3...,American Airlines||American Airlines,CLT||BOS,228||728,251.1
8,2022-04-16,2022-04-17,ATL,BOS,PT5H32M,False,False,956,2022-04-17T07:00:00.000-04:00||2022-04-17T10:2...,2022-04-17T08:21:00.000-04:00||2022-04-17T12:3...,American Airlines||American Airlines,CLT||BOS,228||728,251.1
9,2022-04-16,2022-04-17,ATL,BOS,PT6H38M,False,False,956,2022-04-17T07:00:00.000-04:00||2022-04-17T11:2...,2022-04-17T08:21:00.000-04:00||2022-04-17T13:3...,American Airlines||American Airlines,CLT||BOS,228||728,251.1


In [47]:
flight_prices_processed.filter(
    (~col("segmentsArrivalAirportCode").like("%||%"))
).count()

22066888

In [48]:
from pyspark.sql.functions import col, split

split_airline_names = split(col("segmentsAirlineName"), "\|\|")

flight_prices_processed.filter((split_airline_names[0] != split_airline_names[1])).count()

6757683

In [49]:
flight_prices_processed.count()

82138753

In [50]:
flight_prices_processed_2 = flight_prices_processed

In [51]:
# Define the list of columns to process
columns_to_process = [
    "segmentsDepartureTimeRaw",
    "segmentsAirlineName",
    "segmentsArrivalAirportCode"
]

# Loop through the columns and extract the first value before the first pipeline
for column in columns_to_process:
    flight_prices_processed_2 = flight_prices_processed_2.withColumn(
        f"first_{column}",
        split(col(column), "\|\|")[0]
    )

In [52]:
flight_prices_processed_2.limit(10).toPandas().head(10)

,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isRefundable,isNonStop,totalTravelDistance,segmentsDepartureTimeRaw,segmentsArrivalTimeRaw,segmentsAirlineName,segmentsArrivalAirportCode,segmentsDistance,totalFare,first_segmentsDepartureTimeRaw,first_segmentsAirlineName,first_segmentsArrivalAirportCode
0,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,True,947,2022-04-17T12:57:00.000-04:00,2022-04-17T15:26:00.000-04:00,Delta,BOS,947,248.6,2022-04-17T12:57:00.000-04:00,Delta,BOS
1,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,True,947,2022-04-17T06:30:00.000-04:00,2022-04-17T09:00:00.000-04:00,Delta,BOS,947,248.6,2022-04-17T06:30:00.000-04:00,Delta,BOS
2,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,True,947,2022-04-17T11:35:00.000-04:00,2022-04-17T14:05:00.000-04:00,Delta,BOS,947,248.6,2022-04-17T11:35:00.000-04:00,Delta,BOS
3,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,True,947,2022-04-17T13:59:00.000-04:00,2022-04-17T16:31:00.000-04:00,Delta,BOS,947,248.6,2022-04-17T13:59:00.000-04:00,Delta,BOS
4,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,True,947,2022-04-17T09:59:00.000-04:00,2022-04-17T12:33:00.000-04:00,Delta,BOS,947,248.6,2022-04-17T09:59:00.000-04:00,Delta,BOS
5,2022-04-16,2022-04-17,ATL,BOS,PT2H38M,False,True,947,2022-04-17T10:45:00.000-04:00,2022-04-17T13:23:00.000-04:00,JetBlue Airways,BOS,947,248.6,2022-04-17T10:45:00.000-04:00,JetBlue Airways,BOS
6,2022-04-16,2022-04-17,ATL,BOS,PT4H12M,False,False,956,2022-04-17T08:20:00.000-04:00||2022-04-17T10:2...,2022-04-17T09:50:00.000-04:00||2022-04-17T12:3...,American Airlines||American Airlines,CLT||BOS,228||728,251.1,2022-04-17T08:20:00.000-04:00,American Airlines,CLT
7,2022-04-16,2022-04-17,ATL,BOS,PT5H18M,False,False,956,2022-04-17T08:20:00.000-04:00||2022-04-17T11:2...,2022-04-17T09:50:00.000-04:00||2022-04-17T13:3...,American Airlines||American Airlines,CLT||BOS,228||728,251.1,2022-04-17T08:20:00.000-04:00,American Airlines,CLT
8,2022-04-16,2022-04-17,ATL,BOS,PT5H32M,False,False,956,2022-04-17T07:00:00.000-04:00||2022-04-17T10:2...,2022-04-17T08:21:00.000-04:00||2022-04-17T12:3...,American Airlines||American Airlines,CLT||BOS,228||728,251.1,2022-04-17T07:00:00.000-04:00,American Airlines,CLT
9,2022-04-16,2022-04-17,ATL,BOS,PT6H38M,False,False,956,2022-04-17T07:00:00.000-04:00||2022-04-17T11:2...,2022-04-17T08:21:00.000-04:00||2022-04-17T13:3...,American Airlines||American Airlines,CLT||BOS,228||728,251.1,2022-04-17T07:00:00.000-04:00,American Airlines,CLT


In [53]:
flight_prices_processed_2.select(col("first_segmentsAirlineName")).distinct().toPandas().head(100)

,first_segmentsAirlineName
0,Delta
1,Boutique Air
2,Key Lime Air
3,Sun Country Airlines
4,Contour Airlines
5,United
6,Cape Air
7,Frontier Airlines
8,American Airlines
9,JetBlue Airways


In [54]:
from pyspark.sql.functions import when
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define the mapping of left values to right values
airline_mapping = {
    "Delta": "Delta Air Lines Inc.",
    "Boutique Air": "Compass Airlines",
    "Key Lime Air": "Peninsula Airways Inc.",
    "Sun Country Airlines": "SkyWest Airlines Inc.",
    "Contour Airlines": "Mesa Airlines Inc.",
    "United": "United Air Lines Inc.",
    "Cape Air": "Cape Air",  # If "Cape Air" should remain unchanged
    "Frontier Airlines": "Frontier Airlines Inc.",
    "American Airlines": "American Airlines Inc.",
    "JetBlue Airways": "JetBlue Airways",
    "Alaska Airlines": "Alaska Airlines Inc.",
    "Spirit Airlines": "Spirit Airlines Inc.",
    "Southern Airways Express": "Southwest Airlines Co.",
    "Hawaiian Airlines": "Hawaiian Airlines Inc."
}

# Define a function to map values using the airline_mapping dictionary
def map_airline_name(airline):
    return airline_mapping.get(airline, airline)

# Register the function as a UDF
map_airline_udf = udf(map_airline_name, StringType())

# Apply the UDF to the DataFrame column
flight_prices_processed_2 = flight_prices_processed_2.withColumn('first_segmentsAirlineName', map_airline_udf('first_segmentsAirlineName'))

In [55]:
flight_prices_processed_2.select(col("first_segmentsAirlineName")).distinct().toPandas().head(100)

,first_segmentsAirlineName
0,SkyWest Airlines Inc.
1,United Air Lines Inc.
2,Compass Airlines
3,Peninsula Airways Inc.
4,Frontier Airlines Inc.
5,Cape Air
6,Southwest Airlines Co.
7,Spirit Airlines Inc.
8,JetBlue Airways
9,Alaska Airlines Inc.


In [56]:
columns_to_drop = [
    "segmentsDepartureTimeRaw",
    "segmentsAirlineName",
    'first_segmentsArrivalAirportCode',
    'segmentsArrivalAirportCode'
]

# Drop the specified columns
flight_prices_processed_2 = flight_prices_processed_2.drop(*columns_to_drop)

In [57]:
flight_prices_processed_2.limit(10).toPandas().head(10)

,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isRefundable,isNonStop,totalTravelDistance,segmentsArrivalTimeRaw,segmentsDistance,totalFare,first_segmentsDepartureTimeRaw,first_segmentsAirlineName
0,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,True,947,2022-04-17T15:26:00.000-04:00,947,248.6,2022-04-17T12:57:00.000-04:00,Delta Air Lines Inc.
1,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,True,947,2022-04-17T09:00:00.000-04:00,947,248.6,2022-04-17T06:30:00.000-04:00,Delta Air Lines Inc.
2,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,True,947,2022-04-17T14:05:00.000-04:00,947,248.6,2022-04-17T11:35:00.000-04:00,Delta Air Lines Inc.
3,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,True,947,2022-04-17T16:31:00.000-04:00,947,248.6,2022-04-17T13:59:00.000-04:00,Delta Air Lines Inc.
4,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,True,947,2022-04-17T12:33:00.000-04:00,947,248.6,2022-04-17T09:59:00.000-04:00,Delta Air Lines Inc.
5,2022-04-16,2022-04-17,ATL,BOS,PT2H38M,False,True,947,2022-04-17T13:23:00.000-04:00,947,248.6,2022-04-17T10:45:00.000-04:00,JetBlue Airways
6,2022-04-16,2022-04-17,ATL,BOS,PT4H12M,False,False,956,2022-04-17T09:50:00.000-04:00||2022-04-17T12:3...,228||728,251.1,2022-04-17T08:20:00.000-04:00,American Airlines Inc.
7,2022-04-16,2022-04-17,ATL,BOS,PT5H18M,False,False,956,2022-04-17T09:50:00.000-04:00||2022-04-17T13:3...,228||728,251.1,2022-04-17T08:20:00.000-04:00,American Airlines Inc.
8,2022-04-16,2022-04-17,ATL,BOS,PT5H32M,False,False,956,2022-04-17T08:21:00.000-04:00||2022-04-17T12:3...,228||728,251.1,2022-04-17T07:00:00.000-04:00,American Airlines Inc.
9,2022-04-16,2022-04-17,ATL,BOS,PT6H38M,False,False,956,2022-04-17T08:21:00.000-04:00||2022-04-17T13:3...,228||728,251.1,2022-04-17T07:00:00.000-04:00,American Airlines Inc.


In [58]:
from pyspark.sql.functions import size
# flight_prices_processed_2 = flight_prices_processed_2.withColumn("first_segment_Arrivaltime", split(col("segmentsArrivalTimeRaw"), "\|\|")[-1])
flight_prices_processed_2 = flight_prices_processed_2.withColumn(
    "last_segment_arrival_time",
    split(col("segmentsArrivalTimeRaw"), "\|\|")[size(split(col("segmentsArrivalTimeRaw"), "\|\|"))-1]
)

flight_prices_processed_2.limit(10).toPandas().head(10)

,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isRefundable,isNonStop,totalTravelDistance,segmentsArrivalTimeRaw,segmentsDistance,totalFare,first_segmentsDepartureTimeRaw,first_segmentsAirlineName,last_segment_arrival_time
0,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,True,947,2022-04-17T15:26:00.000-04:00,947,248.6,2022-04-17T12:57:00.000-04:00,Delta Air Lines Inc.,2022-04-17T15:26:00.000-04:00
1,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,True,947,2022-04-17T09:00:00.000-04:00,947,248.6,2022-04-17T06:30:00.000-04:00,Delta Air Lines Inc.,2022-04-17T09:00:00.000-04:00
2,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,True,947,2022-04-17T14:05:00.000-04:00,947,248.6,2022-04-17T11:35:00.000-04:00,Delta Air Lines Inc.,2022-04-17T14:05:00.000-04:00
3,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,True,947,2022-04-17T16:31:00.000-04:00,947,248.6,2022-04-17T13:59:00.000-04:00,Delta Air Lines Inc.,2022-04-17T16:31:00.000-04:00
4,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,True,947,2022-04-17T12:33:00.000-04:00,947,248.6,2022-04-17T09:59:00.000-04:00,Delta Air Lines Inc.,2022-04-17T12:33:00.000-04:00
5,2022-04-16,2022-04-17,ATL,BOS,PT2H38M,False,True,947,2022-04-17T13:23:00.000-04:00,947,248.6,2022-04-17T10:45:00.000-04:00,JetBlue Airways,2022-04-17T13:23:00.000-04:00
6,2022-04-16,2022-04-17,ATL,BOS,PT4H12M,False,False,956,2022-04-17T09:50:00.000-04:00||2022-04-17T12:3...,228||728,251.1,2022-04-17T08:20:00.000-04:00,American Airlines Inc.,2022-04-17T12:32:00.000-04:00
7,2022-04-16,2022-04-17,ATL,BOS,PT5H18M,False,False,956,2022-04-17T09:50:00.000-04:00||2022-04-17T13:3...,228||728,251.1,2022-04-17T08:20:00.000-04:00,American Airlines Inc.,2022-04-17T13:38:00.000-04:00
8,2022-04-16,2022-04-17,ATL,BOS,PT5H32M,False,False,956,2022-04-17T08:21:00.000-04:00||2022-04-17T12:3...,228||728,251.1,2022-04-17T07:00:00.000-04:00,American Airlines Inc.,2022-04-17T12:32:00.000-04:00
9,2022-04-16,2022-04-17,ATL,BOS,PT6H38M,False,False,956,2022-04-17T08:21:00.000-04:00||2022-04-17T13:3...,228||728,251.1,2022-04-17T07:00:00.000-04:00,American Airlines Inc.,2022-04-17T13:38:00.000-04:00


In [59]:
columns_to_drop = [
    "segmentsArrivalTimeRaw",
]

# Drop the specified columns
flight_prices_processed_2 = flight_prices_processed_2.drop(*columns_to_drop)

In [60]:
flight_prices_processed_2.limit(10).toPandas().head(10)

,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isRefundable,isNonStop,totalTravelDistance,segmentsDistance,totalFare,first_segmentsDepartureTimeRaw,first_segmentsAirlineName,last_segment_arrival_time
0,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,True,947,947,248.6,2022-04-17T12:57:00.000-04:00,Delta Air Lines Inc.,2022-04-17T15:26:00.000-04:00
1,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,True,947,947,248.6,2022-04-17T06:30:00.000-04:00,Delta Air Lines Inc.,2022-04-17T09:00:00.000-04:00
2,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,True,947,947,248.6,2022-04-17T11:35:00.000-04:00,Delta Air Lines Inc.,2022-04-17T14:05:00.000-04:00
3,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,True,947,947,248.6,2022-04-17T13:59:00.000-04:00,Delta Air Lines Inc.,2022-04-17T16:31:00.000-04:00
4,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,True,947,947,248.6,2022-04-17T09:59:00.000-04:00,Delta Air Lines Inc.,2022-04-17T12:33:00.000-04:00
5,2022-04-16,2022-04-17,ATL,BOS,PT2H38M,False,True,947,947,248.6,2022-04-17T10:45:00.000-04:00,JetBlue Airways,2022-04-17T13:23:00.000-04:00
6,2022-04-16,2022-04-17,ATL,BOS,PT4H12M,False,False,956,228||728,251.1,2022-04-17T08:20:00.000-04:00,American Airlines Inc.,2022-04-17T12:32:00.000-04:00
7,2022-04-16,2022-04-17,ATL,BOS,PT5H18M,False,False,956,228||728,251.1,2022-04-17T08:20:00.000-04:00,American Airlines Inc.,2022-04-17T13:38:00.000-04:00
8,2022-04-16,2022-04-17,ATL,BOS,PT5H32M,False,False,956,228||728,251.1,2022-04-17T07:00:00.000-04:00,American Airlines Inc.,2022-04-17T12:32:00.000-04:00
9,2022-04-16,2022-04-17,ATL,BOS,PT6H38M,False,False,956,228||728,251.1,2022-04-17T07:00:00.000-04:00,American Airlines Inc.,2022-04-17T13:38:00.000-04:00


In [61]:
columns_to_drop = [
    "segmentsDistance",
]

# Drop the specified columns
flight_prices_processed_2 = flight_prices_processed_2.drop(*columns_to_drop)

In [62]:
flight_prices_processed_2.limit(10).toPandas().head(10)

,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isRefundable,isNonStop,totalTravelDistance,totalFare,first_segmentsDepartureTimeRaw,first_segmentsAirlineName,last_segment_arrival_time
0,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,True,947,248.6,2022-04-17T12:57:00.000-04:00,Delta Air Lines Inc.,2022-04-17T15:26:00.000-04:00
1,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,True,947,248.6,2022-04-17T06:30:00.000-04:00,Delta Air Lines Inc.,2022-04-17T09:00:00.000-04:00
2,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,True,947,248.6,2022-04-17T11:35:00.000-04:00,Delta Air Lines Inc.,2022-04-17T14:05:00.000-04:00
3,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,True,947,248.6,2022-04-17T13:59:00.000-04:00,Delta Air Lines Inc.,2022-04-17T16:31:00.000-04:00
4,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,True,947,248.6,2022-04-17T09:59:00.000-04:00,Delta Air Lines Inc.,2022-04-17T12:33:00.000-04:00
5,2022-04-16,2022-04-17,ATL,BOS,PT2H38M,False,True,947,248.6,2022-04-17T10:45:00.000-04:00,JetBlue Airways,2022-04-17T13:23:00.000-04:00
6,2022-04-16,2022-04-17,ATL,BOS,PT4H12M,False,False,956,251.1,2022-04-17T08:20:00.000-04:00,American Airlines Inc.,2022-04-17T12:32:00.000-04:00
7,2022-04-16,2022-04-17,ATL,BOS,PT5H18M,False,False,956,251.1,2022-04-17T08:20:00.000-04:00,American Airlines Inc.,2022-04-17T13:38:00.000-04:00
8,2022-04-16,2022-04-17,ATL,BOS,PT5H32M,False,False,956,251.1,2022-04-17T07:00:00.000-04:00,American Airlines Inc.,2022-04-17T12:32:00.000-04:00
9,2022-04-16,2022-04-17,ATL,BOS,PT6H38M,False,False,956,251.1,2022-04-17T07:00:00.000-04:00,American Airlines Inc.,2022-04-17T13:38:00.000-04:00


In [63]:
from pyspark.sql.functions import weekofyear, to_date

flight_prices_processed_2 = flight_prices_processed_2.withColumn("searchWeek", weekofyear("searchDate"))
flight_prices_processed_2 = flight_prices_processed_2.withColumn("flightWeek", weekofyear("flightDate"))
flight_prices_processed_2.limit(100).toPandas().head(100)

,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isRefundable,isNonStop,totalTravelDistance,totalFare,first_segmentsDepartureTimeRaw,first_segmentsAirlineName,last_segment_arrival_time,searchWeek,flightWeek
0,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,True,947.0,248.60,2022-04-17T12:57:00.000-04:00,Delta Air Lines Inc.,2022-04-17T15:26:00.000-04:00,15,15
1,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,True,947.0,248.60,2022-04-17T06:30:00.000-04:00,Delta Air Lines Inc.,2022-04-17T09:00:00.000-04:00,15,15
2,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,True,947.0,248.60,2022-04-17T11:35:00.000-04:00,Delta Air Lines Inc.,2022-04-17T14:05:00.000-04:00,15,15
3,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,True,947.0,248.60,2022-04-17T13:59:00.000-04:00,Delta Air Lines Inc.,2022-04-17T16:31:00.000-04:00,15,15
4,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,True,947.0,248.60,2022-04-17T09:59:00.000-04:00,Delta Air Lines Inc.,2022-04-17T12:33:00.000-04:00,15,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022-04-16,2022-04-17,ATL,DFW,PT2H30M,False,True,NaN,168.59,2022-04-17T21:57:00.000-04:00,Spirit Airlines Inc.,2022-04-17T23:27:00.000-05:00,15,15
96,2022-04-16,2022-04-17,ATL,DFW,PT2H13M,False,True,725.0,202.60,2022-04-17T08:30:00.000-04:00,Delta Air Lines Inc.,2022-04-17T09:43:00.000-05:00,15,15
97,2022-04-16,2022-04-17,ATL,DFW,PT2H15M,False,True,725.0,202.60,2022-04-17T16:39:00.000-04:00,Delta Air Lines Inc.,2022-04-17T17:54:00.000-05:00,15,15
98,2022-04-16,2022-04-17,ATL,DFW,PT2H16M,False,True,725.0,202.60,2022-04-17T14:57:00.000-04:00,Delta Air Lines Inc.,2022-04-17T16:13:00.000-05:00,15,15


In [64]:
columns_to_drop = [
    "searchDate",
    "flightDate",
]

# Drop the specified columns
flight_prices_processed_2 = flight_prices_processed_2.drop(*columns_to_drop)

In [65]:
flight_prices_processed_2.limit(100).toPandas().head(100)

,startingAirport,destinationAirport,travelDuration,isRefundable,isNonStop,totalTravelDistance,totalFare,first_segmentsDepartureTimeRaw,first_segmentsAirlineName,last_segment_arrival_time,searchWeek,flightWeek
0,ATL,BOS,PT2H29M,False,True,947.0,248.60,2022-04-17T12:57:00.000-04:00,Delta Air Lines Inc.,2022-04-17T15:26:00.000-04:00,15,15
1,ATL,BOS,PT2H30M,False,True,947.0,248.60,2022-04-17T06:30:00.000-04:00,Delta Air Lines Inc.,2022-04-17T09:00:00.000-04:00,15,15
2,ATL,BOS,PT2H30M,False,True,947.0,248.60,2022-04-17T11:35:00.000-04:00,Delta Air Lines Inc.,2022-04-17T14:05:00.000-04:00,15,15
3,ATL,BOS,PT2H32M,False,True,947.0,248.60,2022-04-17T13:59:00.000-04:00,Delta Air Lines Inc.,2022-04-17T16:31:00.000-04:00,15,15
4,ATL,BOS,PT2H34M,False,True,947.0,248.60,2022-04-17T09:59:00.000-04:00,Delta Air Lines Inc.,2022-04-17T12:33:00.000-04:00,15,15
...,...,...,...,...,...,...,...,...,...,...,...,...
95,ATL,DFW,PT2H30M,False,True,NaN,168.59,2022-04-17T21:57:00.000-04:00,Spirit Airlines Inc.,2022-04-17T23:27:00.000-05:00,15,15
96,ATL,DFW,PT2H13M,False,True,725.0,202.60,2022-04-17T08:30:00.000-04:00,Delta Air Lines Inc.,2022-04-17T09:43:00.000-05:00,15,15
97,ATL,DFW,PT2H15M,False,True,725.0,202.60,2022-04-17T16:39:00.000-04:00,Delta Air Lines Inc.,2022-04-17T17:54:00.000-05:00,15,15
98,ATL,DFW,PT2H16M,False,True,725.0,202.60,2022-04-17T14:57:00.000-04:00,Delta Air Lines Inc.,2022-04-17T16:13:00.000-05:00,15,15


In [66]:
# Define a UDF (User Defined Function) to convert duration strings to minutes
def duration_to_minutes(duration_str):
    try:
        parts = duration_str.split('T')
        parts_l = parts[0].split('P')
        days = int(parts_l[1][0])
    except:
        days = 0

    try:
        parts = duration_str.split('T')
        parts_r = parts[1].split('H')
        hours = int(parts_r[0]) if len(parts_r) > 1 else 0
    except:
        hours = 0

    try:
        parts = duration_str.split('T')
        parts_r = parts[1].split('H')
        minutes = int(parts_r[-1][:-1])
    except:
        minutes = 0

    try:
        return ((24*days) + hours) * 60 + minutes
    except:
        return 0                 

In [67]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

duration_to_minutes_udf = udf(duration_to_minutes, IntegerType())

In [68]:
flight_prices_processed_2 = flight_prices_processed_2.withColumn(
    "travelDurationMinutes",
    duration_to_minutes_udf("travelDuration")
)

In [69]:
flight_prices_processed_2.limit(1).toPandas().head(1)

,startingAirport,destinationAirport,travelDuration,isRefundable,isNonStop,totalTravelDistance,totalFare,first_segmentsDepartureTimeRaw,first_segmentsAirlineName,last_segment_arrival_time,searchWeek,flightWeek,travelDurationMinutes
0,ATL,BOS,PT2H29M,False,True,947,248.6,2022-04-17T12:57:00.000-04:00,Delta Air Lines Inc.,2022-04-17T15:26:00.000-04:00,15,15,149


In [70]:
columns_to_drop = [
    "travelDuration",
]

# Drop the specified columns
flight_prices_processed_2 = flight_prices_processed_2.drop(*columns_to_drop)

In [71]:
flight_prices_processed_2.limit(20).toPandas().head(20)

,startingAirport,destinationAirport,isRefundable,isNonStop,totalTravelDistance,totalFare,first_segmentsDepartureTimeRaw,first_segmentsAirlineName,last_segment_arrival_time,searchWeek,flightWeek,travelDurationMinutes
0,ATL,BOS,False,True,947.0,248.60,2022-04-17T12:57:00.000-04:00,Delta Air Lines Inc.,2022-04-17T15:26:00.000-04:00,15,15,149
1,ATL,BOS,False,True,947.0,248.60,2022-04-17T06:30:00.000-04:00,Delta Air Lines Inc.,2022-04-17T09:00:00.000-04:00,15,15,150
2,ATL,BOS,False,True,947.0,248.60,2022-04-17T11:35:00.000-04:00,Delta Air Lines Inc.,2022-04-17T14:05:00.000-04:00,15,15,150
3,ATL,BOS,False,True,947.0,248.60,2022-04-17T13:59:00.000-04:00,Delta Air Lines Inc.,2022-04-17T16:31:00.000-04:00,15,15,152
4,ATL,BOS,False,True,947.0,248.60,2022-04-17T09:59:00.000-04:00,Delta Air Lines Inc.,2022-04-17T12:33:00.000-04:00,15,15,154
5,ATL,BOS,False,True,947.0,248.60,2022-04-17T10:45:00.000-04:00,JetBlue Airways,2022-04-17T13:23:00.000-04:00,15,15,158
6,ATL,BOS,False,False,956.0,251.10,2022-04-17T08:20:00.000-04:00,American Airlines Inc.,2022-04-17T12:32:00.000-04:00,15,15,252
7,ATL,BOS,False,False,956.0,251.10,2022-04-17T08:20:00.000-04:00,American Airlines Inc.,2022-04-17T13:38:00.000-04:00,15,15,318
8,ATL,BOS,False,False,956.0,251.10,2022-04-17T07:00:00.000-04:00,American Airlines Inc.,2022-04-17T12:32:00.000-04:00,15,15,332
9,ATL,BOS,False,False,956.0,251.10,2022-04-17T07:00:00.000-04:00,American Airlines Inc.,2022-04-17T13:38:00.000-04:00,15,15,398


In [72]:
from pyspark.sql.functions import col, expr, hour, from_utc_timestamp
from pyspark.sql.types import StringType

# Assuming df is your DataFrame containing the column "first_segmentsDepartureTimeRaw"

# Convert timestamp from UTC to local timezone
local_timestamp = from_utc_timestamp("first_segmentsDepartureTimeRaw", "UTC")

# Extract the hour of the day from the local timestamp
hour_of_day = hour(local_timestamp)

# Define a UDF to map the hour of the day to the corresponding daypart
def get_daypart(hour):
    hour = hour-4
    if 16 <= hour < 19:
        return "evening"
    elif 6 <= hour < 12:
        return "morning"
    elif 12 <= hour < 16:
        return "afternoon"
    else:
        return "night"

# Register the UDF with SparkSession
get_daypart_udf = udf(get_daypart, StringType())

# Apply the UDF to create a new column with the daypart
flight_prices_processed_2 = flight_prices_processed_2.withColumn(
    "segmentsDepartureDaypart",
    get_daypart_udf(hour_of_day)
)


In [73]:
from pyspark.sql.functions import col, expr, hour, from_utc_timestamp
from pyspark.sql.types import StringType

# Assuming df is your DataFrame containing the column "first_segmentsDepartureTimeRaw"

# Convert timestamp from UTC to local timezone
local_timestamp = from_utc_timestamp("last_segment_arrival_time", "UTC")

# Extract the hour of the day from the local timestamp
hour_of_day = hour(local_timestamp)


# Apply the UDF to create a new column with the daypart
flight_prices_processed_2 = flight_prices_processed_2.withColumn(
    "segmentArrivalDaypart",
    get_daypart_udf(hour_of_day)
)


In [74]:
flight_prices_processed_2.limit(20).toPandas().head(20)

,startingAirport,destinationAirport,isRefundable,isNonStop,totalTravelDistance,totalFare,first_segmentsDepartureTimeRaw,first_segmentsAirlineName,last_segment_arrival_time,searchWeek,flightWeek,travelDurationMinutes,segmentsDepartureDaypart,segmentArrivalDaypart
0,ATL,BOS,False,True,947.0,248.60,2022-04-17T12:57:00.000-04:00,Delta Air Lines Inc.,2022-04-17T15:26:00.000-04:00,15,15,149,morning,morning
1,ATL,BOS,False,True,947.0,248.60,2022-04-17T06:30:00.000-04:00,Delta Air Lines Inc.,2022-04-17T09:00:00.000-04:00,15,15,150,night,night
2,ATL,BOS,False,True,947.0,248.60,2022-04-17T11:35:00.000-04:00,Delta Air Lines Inc.,2022-04-17T14:05:00.000-04:00,15,15,150,morning,morning
3,ATL,BOS,False,True,947.0,248.60,2022-04-17T13:59:00.000-04:00,Delta Air Lines Inc.,2022-04-17T16:31:00.000-04:00,15,15,152,morning,afternoon
4,ATL,BOS,False,True,947.0,248.60,2022-04-17T09:59:00.000-04:00,Delta Air Lines Inc.,2022-04-17T12:33:00.000-04:00,15,15,154,night,morning
5,ATL,BOS,False,True,947.0,248.60,2022-04-17T10:45:00.000-04:00,JetBlue Airways,2022-04-17T13:23:00.000-04:00,15,15,158,morning,morning
6,ATL,BOS,False,False,956.0,251.10,2022-04-17T08:20:00.000-04:00,American Airlines Inc.,2022-04-17T12:32:00.000-04:00,15,15,252,night,morning
7,ATL,BOS,False,False,956.0,251.10,2022-04-17T08:20:00.000-04:00,American Airlines Inc.,2022-04-17T13:38:00.000-04:00,15,15,318,night,morning
8,ATL,BOS,False,False,956.0,251.10,2022-04-17T07:00:00.000-04:00,American Airlines Inc.,2022-04-17T12:32:00.000-04:00,15,15,332,night,morning
9,ATL,BOS,False,False,956.0,251.10,2022-04-17T07:00:00.000-04:00,American Airlines Inc.,2022-04-17T13:38:00.000-04:00,15,15,398,night,morning


In [75]:
columns_to_drop = [
    "last_segment_arrival_time",
    "first_segmentsDepartureTimeRaw"
]

# Drop the specified columns
flight_prices_processed_2 = flight_prices_processed_2.drop(*columns_to_drop)

In [76]:
flight_prices_processed_2.limit(20).toPandas().head(20)

,startingAirport,destinationAirport,isRefundable,isNonStop,totalTravelDistance,totalFare,first_segmentsAirlineName,searchWeek,flightWeek,travelDurationMinutes,segmentsDepartureDaypart,segmentArrivalDaypart
0,ATL,BOS,False,True,947.0,248.60,Delta Air Lines Inc.,15,15,149,morning,morning
1,ATL,BOS,False,True,947.0,248.60,Delta Air Lines Inc.,15,15,150,night,night
2,ATL,BOS,False,True,947.0,248.60,Delta Air Lines Inc.,15,15,150,morning,morning
3,ATL,BOS,False,True,947.0,248.60,Delta Air Lines Inc.,15,15,152,morning,afternoon
4,ATL,BOS,False,True,947.0,248.60,Delta Air Lines Inc.,15,15,154,night,morning
5,ATL,BOS,False,True,947.0,248.60,JetBlue Airways,15,15,158,morning,morning
6,ATL,BOS,False,False,956.0,251.10,American Airlines Inc.,15,15,252,night,morning
7,ATL,BOS,False,False,956.0,251.10,American Airlines Inc.,15,15,318,night,morning
8,ATL,BOS,False,False,956.0,251.10,American Airlines Inc.,15,15,332,night,morning
9,ATL,BOS,False,False,956.0,251.10,American Airlines Inc.,15,15,398,night,morning


In [77]:
from pyspark.sql.functions import when

# Assuming df is your DataFrame

# Convert boolean values to integers (1 for True, 0 for False)
flight_prices_processed_2 = flight_prices_processed_2.withColumn("isRefundable", when(col("isRefundable"), 1).otherwise(0))
flight_prices_processed_2 = flight_prices_processed_2.withColumn("isNonStop", when(col("isNonStop"), 1).otherwise(0))


In [78]:
from pyspark.sql.functions import col, isnull
null_count = flight_prices_processed_2.filter(isnull(col("totalTravelDistance"))).count()

# Print the count of null values in col1
print("Number of null values in col1:", null_count)

Number of null values in col1: 6094532


In [79]:
from pyspark.sql.functions import col, isnull
flight_prices_processed_2 = flight_prices_processed_2.filter(~isnull(col("totalTravelDistance")))

flight_prices_processed_2.limit(20).toPandas().head(20)

,startingAirport,destinationAirport,isRefundable,isNonStop,totalTravelDistance,totalFare,first_segmentsAirlineName,searchWeek,flightWeek,travelDurationMinutes,segmentsDepartureDaypart,segmentArrivalDaypart
0,ATL,BOS,0,1,947,248.60,Delta Air Lines Inc.,15,15,149,morning,morning
1,ATL,BOS,0,1,947,248.60,Delta Air Lines Inc.,15,15,150,night,night
2,ATL,BOS,0,1,947,248.60,Delta Air Lines Inc.,15,15,150,morning,morning
3,ATL,BOS,0,1,947,248.60,Delta Air Lines Inc.,15,15,152,morning,afternoon
4,ATL,BOS,0,1,947,248.60,Delta Air Lines Inc.,15,15,154,night,morning
5,ATL,BOS,0,1,947,248.60,JetBlue Airways,15,15,158,morning,morning
6,ATL,BOS,0,0,956,251.10,American Airlines Inc.,15,15,252,night,morning
7,ATL,BOS,0,0,956,251.10,American Airlines Inc.,15,15,318,night,morning
8,ATL,BOS,0,0,956,251.10,American Airlines Inc.,15,15,332,night,morning
9,ATL,BOS,0,0,956,251.10,American Airlines Inc.,15,15,398,night,morning


In [80]:
flight_prices_processed_2.columns

['startingAirport',
 'destinationAirport',
 'isRefundable',
 'isNonStop',
 'totalTravelDistance',
 'totalFare',
 'first_segmentsAirlineName',
 'searchWeek',
 'flightWeek',
 'travelDurationMinutes',
 'segmentsDepartureDaypart',
 'segmentArrivalDaypart']

In [81]:
output_path = "/Users/shreyasmac/Documents/Notes/Big Data/Final Project/flight_prices_processed_2"
# Save the DataFrame to a CSV file
flight_prices_processed_2.write.csv(output_path, header=True, mode="overwrite")

travelDurationMinutes might have an issue it was created from the UDF

In [ ]:
ohe_cols = ['startingAirport',
             'destinationAirport',             
             'first_segmentsAirlineName',
             'searchWeek',
             'flightWeek',
             'segmentsDepartureDaypart',
             'segmentArrivalDaypart']

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline


indexers = [StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid="keep") for column in ohe_cols]

# Perform OneHotEncoding on the indexed columns
encoders = [OneHotEncoder(inputCol=column+"_index", outputCol=column+"_onehot") for column in ohe_cols]

# Create a pipeline to execute the indexers and encoders sequentially
pipeline = Pipeline(stages=indexers + encoders)

# Fit the pipeline to the data and transform the DataFrame
model = pipeline.fit(flight_prices_processed_2)
encoded_df = model.transform(flight_prices_processed_2)

# Show the encoded DataFrame
encoded_df.show()


In [ ]:
encoded_df.columns

In [ ]:
columns = encoded_df.columns

# Reorder the columns putting the 5th column at the end
new_columns = columns[:5] + columns[6:] + [columns[5]]

# Select the columns in the new order
encoded_df = encoded_df.select(*new_columns)

In [ ]:
encoded_df.limit(100).toPandas().head(100)

In [ ]:
encoded_df = encoded_df.drop("isNonStop")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Sample 30% of the data randomly without replacement
sample_df = encoded_df.sample(withReplacement=False, fraction=0.1)

# Convert sampled DataFrame to Pandas DataFrame
pandas_sample_df = sample_df.select('totalFare', 'isRefundable').toPandas()

# Plot the relationship between col1 and col2
plt.figure(figsize=(10, 6))
sns.scatterplot(y='totalFare', x='isRefundable', data=pandas_sample_df)
plt.title('Relationship between totalFare and isRefundable (Randomly Sampled)')
plt.xlabel('totalFare')
plt.ylabel('isRefundable')
plt.show()


In [ ]:
encoded_df = encoded_df.drop("isRefundable")

In [ ]:
columns_to_drop = [
    "startingAirport",
    "destinationAirport",
    "first_segmentsAirlineName",
    "searchWeek",
    "flightWeek",
    "segmentsDepartureDaypart",
    "segmentArrivalDaypart",
    "isNonStop"
]

# Drop the specified columns
encoded_df = encoded_df.drop(*columns_to_drop)

In [ ]:
encoded_df.limit(100).toPandas().head(100)

In [ ]:
train_df, test_df = encoded_df.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Combine features into a single vector column
# ["FlightWeek", "Distance", "Airline_onehot", "Origin_onehot", "Dest_onehot"]
trainAssembler = VectorAssembler(inputCols=train_df.columns[:-1],
                                 outputCol="features")
trainOutput = trainAssembler.transform(train_df)

testAssembler = VectorAssembler(inputCols=test_df.columns[:-1],
                                outputCol="features")
testOutput = testAssembler.transform(test_df)

# Create a LinearRegression model
lr = LinearRegression(featuresCol="features", labelCol="totalFare")

# Train the model
model = lr.fit(trainOutput)

# Make predictions
predictions = model.transform(testOutput)


In [ ]:
from pyspark.sql.functions import abs

# Evaluate the model
evaluator = RegressionEvaluator(labelCol="totalFare", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
mae = predictions.select(abs(predictions["totalFare"] - predictions["prediction"]).alias("error")).agg({"error": "mean"}).collect()[0][0]

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

Random Forest

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Combine features into a single vector column
trainAssembler = VectorAssembler(inputCols=train_df.limit(1000).columns[:-1],
                                 outputCol="features")
trainOutput = trainAssembler.transform(train_df)

testAssembler = VectorAssembler(inputCols=test_df.limit(100).columns[:-1],
                                outputCol="features")
testOutput = testAssembler.transform(test_df)

# Create a RandomForestRegressor model
rf = RandomForestRegressor(featuresCol="features", labelCol="totalFare", numTrees=100)

# Train the model
model = rf.fit(trainOutput)

# Make predictions
predictions = model.transform(testOutput)


In [ ]:
from pyspark.sql.functions import abs

# Evaluate the model
evaluator = RegressionEvaluator(labelCol="totalFare", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
mae = predictions.select(abs(predictions["totalFare"] - predictions["prediction"]).alias("error")).agg({"error": "mean"}).collect()[0][0]

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("Mean Absolute Error (MAE) on test data = %g" % mae)